In [138]:
import pandas as pd
import string

In [3]:
lennar = pd.read_json('./lennar-data.json')

In [4]:
lennar.shape

(11365, 185)

In [9]:
lennar.head(1)

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL,Data_Dump,feature_0,...,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175
0,Move-in ready,"$374,990",4.0,3.5,2179,"6417 Mentor Place, Raleigh, NC",Bradley in 5401 North,https://www.lennar.com/new-homes/north-carolin...,"{'Kitchen': ['36"" cabinetry with crown molding...","36"" cabinetry with crown molding and nickel ha...",...,None,None,None,None,None,None,None,None,None,None


In [90]:
clean = lennar.drop_duplicates(subset=['Availability', 'Price', 'Beds', 'Baths', 'Sqft', 'Address', 'Community', 'URL'],
                    keep='first',
                    ignore_index=True)

In [91]:
clean.shape

(9716, 185)

After removing duplicate homes (from the fact that some cities that are close together list the same homes under different urls -- ex Charlotte is under SC and NC for some reason), we see there are **9716 homes in the dataset**

In [183]:
def search_homes(keyword):
    """
    Method that searches through the Data_dump dict structure
    
    Returns:
        vals: dict where keys include keyword and values are counts
        home_count: # of unique homes where the keyword is present
    """
    keyword = keyword.lower()
    vals = dict()
    home_count = 0
    index = 0
    
    for d in clean.Data_Dump:
        index += 1
        lines = [item.lower() for sublist in d.values() for item in sublist]
        cleaned_lines = [l.strip().translate(str.maketrans('', '', string.punctuation)).replace(
            '®', '').replace('\n', '').replace('\r', '') for l in lines]
        cleaned_words = [i for l in cleaned_lines for i in l.split()]
        
        if keyword in cleaned_words:
            home_count += 1
            
            for cl in cleaned_lines:
                if keyword in cl:
                    vals[cl] = vals.get(cl, 0) + 1

    
    return home_count, vals

Okay so here's the function. Try 'usb', 'wifi', 'motion' etc

In [210]:
n, d = search_homes('motion')

In [211]:
n

5981

In [212]:
d

{'ring alarm security kit – ring alarm motion detector¹': 2695,
 'ring alarm security kit  ring alarm motion detector¹': 72,
 'security system with motion sensors at firstfloor windows': 203,
 'ring alarm security kit  ring alarm motion detector': 432,
 'ring alarm security kit – ring alarm motion detector': 1196,
 'ring alarm security kit²  ring alarm motion detector': 33,
 'ring alarm security kit ndash base station keypad motion detector smoke and co listener and window and door contact sensor': 27,
 'ring alarm security kit – base station keypad motion detector smoke and co listener and window and door contact sensor': 15,
 'ring alarm security kit – base station keypad motion detector smoke and co listener window and door contact sensor': 19,
 'ring alarm security kit and base station keypad motion detector smoke and co listener and window and door contact sensor': 6,
 'ring alarm security kit ndash base station keypad motion detector smoke and co listener window and door contact 

In [213]:
sum(d.values())

6143